# NDVI computation with eodag


In this tutorial, we compute a bandmath on a Sentinel-2 source, directly by giving an extent with eodag. The area of interest is the center of Toulouse.

Let's start by setting your personal credentials to access [peps service](https://peps.cnes.fr) by filling your username and password bellow:

In [1]:
import os
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

Then we create a workspace directory where all our files and configuration will live:

In [1]:
from eodag.api.core import EODataAccessGateway
from eodag.utils.logging import setup_logging

workspace = 'eodag_workspace'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
setup_logging(verbose=1)
dag = EODataAccessGateway()

2019-04-15 16:46:36,038-15s eodag.config                     [INFO    ] Loading user configuration from: /tmp/eodag_workspace/eodag_conf.yml


We define an extent and search for the same product type as in the previous tutorial for the entire month of May 2018:

In [2]:
product_type = 'S2_MSI_L1C'
extent = {
    'lonmin': 1.306000,
    'lonmax': 1.551819,
    'latmin': 43.527642,
    'latmax': 43.662905
}

products, estimated_nbr_of_results = dag.search(
    productType=product_type,
    start='2018-05-01',
    end='2018-05-31',
    box=extent,
    cloudCover=1
)
product = products[0]
product.properties['title']

2019-04-15 16:46:41,035-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2019-04-15 16:46:41,037-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?box=1.306,43.527642,1.551819,43.662905&startDate=2018-05-01&cloudCover=[0,1]&productType=S2MSI1C&completionDate=2018-05-31&maxRecords=1&page=1
2019-04-15 16:46:41,751-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?box=1.306,43.527642,1.551819,43.662905&startDate=2018-05-01&cloudCover=[0,1]&productType=S2MSI1C&completionDate=2018-05-31&maxRecords=20&page=1
2019-04-15 16:46:42,205-15s eodag.core                       [INFO    ] Found 1 result(s) on provider 'peps'


u'S2A_MSIL1C_20180511T105031_N0206_R051_T31TCJ_20180511T125821'

The method **get_data** allows us to directly retrieve a subset of the EO product without downloading it if it has already been downloaded. If it has not, it is first downloaded locally and then the subset is retrieved. In the final version of eodag, this retrieval will be done without the entire product downloaded locally.

In [3]:
VIR = product.get_data(crs='epsg:4326', resolution=0.0001, band='B04', extent=(1.435905, 43.586857, 1.458907, 43.603827))
NIR = product.get_data(crs='epsg:4326', resolution=0.0001, band='B08', extent=(1.435905, 43.586857, 1.458907, 43.603827))
NDVI = (NIR - VIR * 1.) / (NIR + VIR)

2019-04-15 16:46:42,241-15s eodag.api.product                [WARNING ] Eodag does not support getting data from distant sources by now. Falling back to first downloading the product and then getting the data...
2019-04-15 16:46:42,245-15s eodag.plugins.download.http      [INFO    ] Download url: https://peps.cnes.fr/resto/collections/S2ST/c17952c9-0697-5bc2-b874-273ba33d328e/download
2019-04-15 16:46:42,251-15s eodag.api.product                [INFO    ] Remote location of the product is still available through its 'remote_location' property: https://peps.cnes.fr/resto/collections/S2ST/c17952c9-0697-5bc2-b874-273ba33d328e/download


Now we can plot the result and see in green the "Prairie des Filtres" on the left, and "Grand-Rond"-"Jardin des plantes" parks on the right center. In red, the "Garonne".

In [4]:
import matplotlib.pyplot as plt

plt.imshow(NDVI, cmap='RdYlGn', aspect='auto')
plt.show()

Finally we can save this picture and share it with who we want !

In [5]:
plt.savefig(os.path.join(workspace, 'ndvi_toulouse.png'))

<Figure size 432x288 with 0 Axes>